<a href="https://colab.research.google.com/github/MungaiMwangi001/LLMs/blob/main/GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load the dataset**

In [21]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("mteb/tweet_sentiment_extraction")
df = pd.DataFrame(dataset['train'])

**TOkenizer**

In [22]:
from transformers import GPT2Tokenizer
from datasets import load_dataset


# Loading the dataset to train our model
dataset = load_dataset("mteb/tweet_sentiment_extraction")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer_config.json
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
  

**Initialize  the base model**

In [23]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# Remove the 'id' and 'text' columns as they are not needed for training and cause issues with the data collator
small_train_dataset = small_train_dataset.remove_columns(["id", "text"])
small_eval_dataset = small_eval_dataset.remove_columns(["id", "text"])

**Initialize the base model**

In [24]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

# Set the padding token ID in the model's configuration
# Use the tokenizer's pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "ta

In [25]:
!pip install evaluate==0.4.2

**Evaluate Method**

In [26]:
import transformers
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score
import numpy as np
import torch
import logging
import os

# Optional: disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Set up logging
logging.basicConfig(level=logging.INFO)
transformers.logging.set_verbosity_info()

print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")

# Calculate steps per epoch
n_gpu = torch.cuda.device_count() or 1
print(f"Using {n_gpu} GPU(s)")

per_device_batch_size = 8  # Increased from 1
grad_accumulation = 1
total_batch_size = per_device_batch_size * n_gpu * grad_accumulation
steps_per_epoch = max(1, len(small_train_dataset) // total_batch_size)
print(f"Steps per epoch: {steps_per_epoch}")

# Compute metrics function using sklearn
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Flatten predictions and labels
    flat_predictions = predictions.flatten()
    flat_labels = np.asarray(labels).flatten()

    # Filter out ignored loss indices (-100)
    valid_indices = flat_labels != -100
    valid_predictions = flat_predictions[valid_indices].astype(int)
    valid_labels = flat_labels[valid_indices].astype(int)

    # Compute accuracy
    return {"accuracy": accuracy_score(valid_labels, valid_predictions)}

training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=per_device_batch_size,
    per_device_eval_batch_size=per_device_batch_size,
    gradient_accumulation_steps=grad_accumulation,
    num_train_epochs=2,
    learning_rate=5e-5,

    # Use the correct parameter name
    eval_strategy="steps",
    eval_steps=steps_per_epoch,

    logging_steps=50,
    save_strategy="epoch",

    remove_unused_columns=False,
    no_cuda=not torch.cuda.is_available(),
    fp16=torch.cuda.is_available(),
    overwrite_output_dir=True,
    report_to="none"
)

# Initialize Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Start training
print("Starting training...")
train_result = trainer.train()
print("Training completed!")

# Save model
trainer.save_model("final_model")
print("Model saved to 'final_model' directory")

# Evaluate
if small_eval_dataset is not None:
    print("Evaluating on validation set...")
    eval_results = trainer.evaluate()
    print(f"Validation results: {eval_results}")


PyTorch: setting up devices
average_tokens_across_devices is True but world size is 1. Setting it to False automatically.
Using auto half precision backend


Transformers version: 4.55.0
PyTorch version: 2.6.0+cu124
Using 1 GPU(s)
Steps per epoch: 125
Starting training...


***** Running training *****
  Num examples = 1,000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 250
  Number of trainable parameters = 124,442,112


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label_text` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

**Finetune   usin the Trainer Method**

**Prepare labels for Token Classification**

The `AutoModelForTokenClassification` model expects a `labels` column in the dataset where each entry is a list of token-level class indices corresponding to the input tokens. You need to create this column based on your original sentiment span annotations.

For example, if your original labels are start and end indices of the sentiment span within the text, you would need to:
1. Align the character-level start and end indices with the tokenized input.
2. Create a list of labels for each token, where each label is an integer representing the class (e.g., 0 for 'O' - outside, 1 for 'B-sentiment' - beginning of sentiment span, 2 for 'I-sentiment' - inside of sentiment span). You might need different classes for different sentiment types (positive, negative, neutral).
3. Handle special tokens (like `[CLS]`, `[SEP]`, padding) by assigning them a special label value (e.g., -100) that the model can ignore during loss calculation.

In [ ]:
# This is a placeholder cell. You need to implement the logic here
# to create the 'labels' column in your tokenized_datasets.

# Example: Function to align labels and create token-level labels
def align_labels_with_tokens(examples):
    tokenized_inputs = tokenizer(examples["text"], padding="max_length", truncation=True)
    labels = []
    for i, label in enumerate(examples["label"]): # Assuming 'label' is the sentiment class index
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # Create token-level labels based on the word ids and original sentiment span
        # This is a simplified example and needs to be adapted to your specific label format
        token_labels = []
        # You need to implement the logic to map original span labels to token labels
        # based on word_ids.
        # For now, let's create dummy labels (e.g., all 0s) to make the code runnable,
        # but this will not train the model correctly for sentiment extraction.
        for word_id in word_ids:
            if word_id is None:
                token_labels.append(-100) # Ignore special tokens
            else:
                # Replace this with your logic to assign the correct token label
                # based on the original sentiment span and the word_id
                token_labels.append(0) # Placeholder: Assign 0 for now

        labels.append(token_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the function to your dataset (replace with your actual dataset object)
# You might need to apply this to both train and test splits
# tokenized_datasets = dataset.map(align_labels_with_tokens, batched=True)

# After creating the 'labels' column, you should verify its structure and content.
# print(tokenized_datasets['train'][0]['labels'])

In [ ]:
# Since the above cell is a placeholder and won't create correct labels for sentiment extraction,
# and to make the Trainer initialization runnable without the 'labels' error,
# let's temporarily add a dummy 'labels' column to the small datasets.
# **IMPORTANT:** This is only for making the code run and will NOT train the model correctly
# for the sentiment extraction task. You MUST replace this with the correct label preparation logic.

# Assuming the tokenized inputs have a certain sequence length (e.g., 512 from truncation=True in tokenize_function)
# You should get the actual sequence length from your tokenized dataset if it's different.
sequence_length = len(small_train_dataset[0]['input_ids']) # Get sequence length from actual data

def add_dummy_labels(examples):
    # Create dummy labels (e.g., all 0s or a random sequence) with the correct shape
    # The shape should be (batch_size, sequence_length) for token classification
    dummy_labels = [[0] * sequence_length for _ in range(len(examples['input_ids']))]
    examples['labels'] = dummy_labels
    return examples

# Apply the dummy label creation to your small datasets
# small_train_dataset = small_train_dataset.map(add_dummy_labels, batched=True)
# small_eval_dataset = small_eval_dataset.map(add_dummy_labels, batched=True)

# **Note:** The above dummy label creation is commented out because applying map again
# might overwrite previous tokenization results or cause issues if applied incorrectly.
# A better approach is to ensure the initial tokenization and labeling function
# correctly creates the 'labels' column.

# Assuming the 'labels' column is now correctly present in small_train_dataset and small_eval_dataset
# after running the appropriate label preparation logic.